In [6]:
import numpy as np
import mne
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os

In [7]:
K.set_image_data_format('channels_last')

# Directory where the data is stored
data_dir = 'data'  # Change this to your actual data directory

In [8]:
# Mapping of run indices to tasks/baselines
run_mapping = {
    1: 'baseline_open',
    2: 'baseline_closed',
    3: 'task_1',
    4: 'task_2',
    5: 'task_3',
    6: 'task_4',
    7: 'task_1',
    8: 'task_2',
    9: 'task_3',
    10: 'task_4',
    11: 'task_1',
    12: 'task_2',
    13: 'task_3',
    14: 'task_4'
}

# Mapping of event IDs to labels
event_id_mapping = {
    'T0': 0,  # Rest
    'T1_left': 1,  # Left fist
    'T1_both': 2,  # Both fists
    'T2_right': 3,  # Right fist
    'T2_both': 4  # Both feet
}

In [9]:
def process_subject(subject_dir):
    edf_files = [f for f in os.listdir(subject_dir) if f.endswith('.edf')]
    X, y = [], []
    max_samples = 0

    for edf_file in edf_files:
        edf_path = os.path.join(subject_dir, edf_file)
        run_index = int(edf_file.split('R')[1].split('.')[0])  # Extract run index from filename
        task_label = run_mapping.get(run_index, 'unknown')

        raw = mne.io.read_raw_edf(edf_path, preload=True)
        raw.notch_filter(freqs=60.0) # US powerline frequency ( Data was collected at New York )
        raw.filter(2, None, method='iir')  # High-pass filter at 2 Hz
        
        # Define event annotations
        events, event_id = mne.events_from_annotations(raw)
        tmin, tmax = 0., 4.1  # Varies between 4.1 to 4.2s

        # Create epochs based on events
        epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False, baseline=None, preload=True)
        labels = []

        for e in epochs.events:
            if task_label in ['task_1', 'task_2']:
                if e[2] == event_id['T1']:
                    labels.append(event_id_mapping['T1_left'])
                elif e[2] == event_id['T2']:
                    labels.append(event_id_mapping['T2_right'])
            elif task_label in ['task_3', 'task_4']:
                if e[2] == event_id['T1']:
                    labels.append(event_id_mapping['T1_both'])
                elif e[2] == event_id['T2']:
                    labels.append(event_id_mapping['T2_both'])
            else:
                labels.append(event_id_mapping['T0'])
        
        max_samples = max(max_samples, epochs.get_data().shape[2])
        X.append(epochs.get_data() * 1000)
        y.append(labels)
    
    return X, y, max_samples

In [10]:
def pad_or_trim(data, target_length):
    """Pad or trim data to the target length."""
    def pad(array):
        if array.shape[2] < target_length:
            pad_width = target_length - array.shape[2]
            return np.pad(array, ((0, 0), (0, 0), (0, pad_width)), mode='constant')
        return array[:, :, :target_length]
    
    return np.concatenate([pad(d) for d in data], axis=0)

In [11]:
def load_data(data_dir):
    subjects = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    X, y = [], []
    max_samples = 0

    for subject in subjects:
        subject_dir = os.path.join(data_dir, subject)
        X_subject, y_subject, subject_max_samples = process_subject(subject_dir)
        max_samples = max(max_samples, subject_max_samples)
        X.append(X_subject)
        y.append(y_subject)

    # Pad or trim all data to the maximum number of samples
    X = [pad_or_trim(x, max_samples) for x in X]
    y = [np.concatenate(ys) for ys in y]
    
    return np.concatenate(X, axis=0), np.concatenate(y, axis=0)

In [12]:
# Load data for all subjects
X, y = load_data(data_dir)

Extracting EDF parameters from c:\temp\BME_442\bme442project\data\S001\S001R01.edf...
EDF file detected


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1057 samples (6.606 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 2 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 2.00 Hz: -6.02 dB

Used Annotations descriptions: ['T0']
Not setting metadata
1 matchi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1057 samples (6.606 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 2 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 2.00 Hz: -6.02 dB

Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
30 matching events found
No baseline correction applied
Using data from preloaded Raw for 30 events and 657 original

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 2 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 2.00 Hz: -6.02 dB

Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
30 matching events found
No baseline correction applied
Using data from preloaded Raw for 30 events and 657 original time points ...
0 bad epochs dropped
Extracting EDF parameters from c:\temp\BME_442\bme442project\data\S035\S035R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Setting up high-pass filter at 2 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 2.00 Hz: -6.02 dB

Used Annotations descriptions: ['T0', 'T1', 'T2']


KeyboardInterrupt: 

In [ ]:
# Define parameters
kernels, chans, samples = 1, X.shape[1], X.shape[2]

In [ ]:
# Split data into train, validate, and test sets (adjust as needed)
train_size = int(0.5 * len(X))
validate_size = int(0.25 * len(X))

In [ ]:
X_train = X[:train_size]
Y_train = y[:train_size]
X_validate = X[train_size:train_size + validate_size]
Y_validate = y[train_size:train_size + validate_size]
X_test = X[train_size + validate_size:]
Y_test = y[train_size + validate_size:]

In [ ]:
# One-hot encode the labels
Y_train = np_utils.to_categorical(Y_train)
Y_validate = np_utils.to_categorical(Y_validate)
Y_test = np_utils.to_categorical(Y_test)

In [ ]:
# Reshape data for EEGNet
X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_validate = X_validate.reshape(X_validate.shape[0], chans, samples, kernels)
X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)

In [ ]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (19784, 64, 321, 1)
19784 train samples
9893 test samples


In [ ]:
# Initialize EEGNet model
model = EEGNet(nb_classes=Y_train.shape[1], Chans=chans, Samples=samples,
               dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16,
               dropoutType='Dropout')

In [ ]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set up model checkpointing
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

# Train the model
fittedModel = model.fit(X_train, Y_train, batch_size=16, epochs=300, verbose=2,
                        validation_data=(X_validate, Y_validate),
                        callbacks=[checkpointer])


Epoch 1/300

Epoch 1: val_loss improved from inf to 1.01695, saving model to /tmp\checkpoint.h5
1237/1237 - 84s - loss: 1.0574 - accuracy: 0.5077 - val_loss: 1.0170 - val_accuracy: 0.5191 - 84s/epoch - 68ms/step
Epoch 2/300


c:\temp\BME_442\bme442project\venv-3.10.5\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.01695 to 0.98133, saving model to /tmp\checkpoint.h5
1237/1237 - 81s - loss: 0.9968 - accuracy: 0.5256 - val_loss: 0.9813 - val_accuracy: 0.5437 - 81s/epoch - 65ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.98133 to 0.96655, saving model to /tmp\checkpoint.h5
1237/1237 - 84s - loss: 0.9806 - accuracy: 0.5407 - val_loss: 0.9666 - val_accuracy: 0.5533 - 84s/epoch - 68ms/step
Epoch 4/300


KeyboardInterrupt: 

In [ ]:
# Load best weights
model.load_weights('/tmp/checkpoint.h5')

In [ ]:
# Evaluate model
probs = model.predict(X_test)
preds = probs.argmax(axis=-1)
acc = np.mean(preds == Y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc))

In [ ]:
# PyRiemann part (for comparison)
n_components = 2
clf = make_pipeline(XdawnCovariances(n_components), TangentSpace(metric='riemann'), LogisticRegression())
X_train_reshaped = X_train.reshape(X_train.shape[0], chans, samples)
X_test_reshaped = X_test.reshape(X_test.shape[0], chans, samples)
clf.fit(X_train_reshaped, Y_train.argmax(axis=-1))
preds_rg = clf.predict(X_test_reshaped)
acc2 = np.mean(preds_rg == Y_test.argmax(axis=-1))
print("Classification accuracy: %f " % (acc2))

In [ ]:
# Plot confusion matrices
names = ['rest', 'left fist', 'right fist', 'both fists', 'both feet']

In [ ]:
cm_eegnet = confusion_matrix(Y_test.argmax(axis=-1), preds)
disp_eegnet = ConfusionMatrixDisplay(confusion_matrix=cm_eegnet, display_labels=names)
disp_eegnet.plot(cmap=plt.cm.Blues)
plt.title('EEGNet-8,2')
plt.show()

In [ ]:
cm_rg = confusion_matrix(Y_test.argmax(axis=-1), preds_rg)
disp_rg = ConfusionMatrixDisplay(confusion_matrix=cm_rg, display_labels=names)
disp_rg.plot(cmap=plt.cm.Blues)
plt.title('xDAWN + RG')
plt.show()